In [5]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [6]:
X = housing["data"]
y = housing["target"]

In [9]:
import pandas as pd
import numpy as np
pd.DataFrame(X)

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [22]:
np.mean(y)

2.068558169089147

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [27]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

F:\Anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR(random_state=42)

In [28]:
from sklearn.metrics import mean_squared_error
y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

In [29]:
np.sqrt(mse)

# not working well

0.9819256687727764

In [30]:
np.sqrt(mse) *np.mean(y)*10000
# too big

20311.703635782505

In [17]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, 
                                   verbose=2,cv=3, random_state=42)
# gammaを reciprocalで指定し、0.001から0.1の間での確率分布

# Cをuniformで指定し、範囲1から10の間で確率分布（一様な値がuniformで指定される）

# n_iterパラメータは、探索するパラメータの組み合わせの数 ここでは10

# verboseパラメータは、探索の詳細なログを出力するかどうかを制御します
# （この例では2で、中間レベルのログが出力されます）
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   4.9s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   4.9s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   5.0s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   4.9s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   4.9s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   4.7s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   4.5s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   4.6s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   4.6s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   4.4s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   4.4s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025EF2C607C0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025EF0410A60>},
                   random_state=42, verbose=2)

In [18]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

In [19]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785369

In [24]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.5929168385528745

In [25]:
np.sqrt(mse) *np.mean(y)*10000

12264.829699790595